### 1. Reading CSV data
#### 1.1 Reading CSV data with an inferred schema

In [27]:
import os
os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-1.8"
# 配置hadoop路径
os.environ['HADOOP_HOME'] = "D:/hadoop-3.3.4"
# 配置python解释器
os.environ['PYSPARK_PYTHON'] = "D:/Anaconda/envs/DDL/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "D:/Anaconda/envs/DDL/python.exe"
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = (SparkSession
         .builder
         .appName("DataLoad")
         .master("local[2]")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

# Set log level to ERROR
spark.sparkContext.setLogLevel("ERROR")
# 获取 SparkContext
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[2] appName=DataLoad>


In [7]:
csv_file_location = "../data/dataload/netflix_titles.csv"

# Read CSV file into a DataFrame
df = (spark.read
      .format("csv")
      .option("header", "true")
      .load(csv_file_location))
# Display contents of DataFrame
df.show(5)


+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|Septem

In [ ]:

# Print schema of DataFrame
df.printSchema()


#### 1.2. Reading CSV data with explicit schema

In [19]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

# Define a Schema
schema = StructType([
    StructField("show_id", StringType(), True),
    StructField("type", StringType(), True),
    StructField("title", StringType(), True),
    StructField("director", StringType(), True),
    StructField("cast", StringType(), True),
    StructField("country", StringType(), True),
    StructField("date_added", DateType(), True),
    StructField("release_year", IntegerType(), True),
    StructField("rating", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("listed_in", StringType(), True),
    StructField("description", StringType(), True)])

# Read CSV file into a DataFrame
df1 = (spark.read.format("csv")
      .option("header", "true")
      .schema(schema)
      .load(csv_file_location))

df1.show(5)


+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|      null|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|      null|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|      null|        2021| TV-MA| 1 Season|Crime

### 2. Reading Json data
#### 2.1 what is json

- JSON文件作为一种轻量级的文本格式, 用于表示结构化数据. JSON 文件通常以 .json 作为文件扩展名, 例如 data.json
#### Basic Structure
- 一个 JSON 文件通常由一个 **JSON 对象**或一个 **JSON 数组**组成. JSON 对象是一组**无序**的键值对, 而 JSON 数组是一组**有序**的值
- JSON对象: `{ "name": "John", "age": 30, "city": "New York" }`
- JSON 数组: `[ "apple", "banana", "orange" ]`
- JSON 允许在对象中嵌套对象, 也允许在数组中嵌套数组或对象

   ```
   { "person": { "name": "Alice", "age": 25,
   "address": { "city": "Paris", "zipCode": "75001" } },
   "fruits": ["apple", "orange", {"type": "banana", "color": "yellow"}] }
   ```

#### 2.2. Reading JSON data with an inferred schema


In [13]:
json_file_location = "../data/dataload/nobel_prizes.json"

# Read JSON file into a DataFrame
df2 = (spark.read.format("json")
      .option("multiLine", "true")
      .load(json_file_location))

df2.printSchema()

root
 |-- category: string (nullable = true)
 |-- laureates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- motivation: string (nullable = true)
 |    |    |-- share: string (nullable = true)
 |    |    |-- surname: string (nullable = true)
 |-- overallMotivation: string (nullable = true)
 |-- year: string (nullable = true)



In [20]:
# Display contents of DataFrame
df2.show(5)

+----------+--------------------+-----------------+----+
|  category|           laureates|overallMotivation|year|
+----------+--------------------+-----------------+----+
| chemistry|[{Carolyn, 1015, ...|             null|2022|
| economics|[{Ben, 1021, "for...|             null|2022|
|literature|[{Annie, 1017, "f...|             null|2022|
|     peace|[{Ales, 1018, "Th...|             null|2022|
|   physics|[{Alain, 1012, "f...|             null|2022|
+----------+--------------------+-----------------+----+
only showing top 5 rows



#### 2.3. `flatten()` , `explode()` and `collect_list()` functions

- `collect_list` 函数将相同组中的列值收集到一个列表中

In [23]:
from pyspark.sql.functions import col
from pyspark.sql.functions import explode
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import flatten


#collect_list("laureates") 会将 laureates 列的所有值按行聚合到一个列表中。
#alias("information") 将结果列重命名为 information
collect_df = df2.select(collect_list("laureates").alias("information"))
collect_df.show()
print("_______________________________________________________________")
# use flatten() function to merge all the elements of the inner arrays
flattened_df = collect_df.select(flatten("information").alias("merged_data"))
flattened_df.show()

+--------------------+
|         information|
+--------------------+
|[[{Carolyn, 1015,...|
+--------------------+

_______________________________________________________________
+--------------------+
|         merged_data|
+--------------------+
|[{Carolyn, 1015, ...|
+--------------------+



 - `explode()` 函数将数组中的每个元素转换为一行, 并将数组中的每个元素作为新行的值

In [25]:

df_flattened = (
    df2
    .withColumn("laureates",explode(col("laureates"))) # Explode the laureates array column into rows
    .select(col("category")
            , col("year")
            , col("overallMotivation")
            , col("laureates.id")
            , col("laureates.firstname")
            , col("laureates.surname")
            , col("laureates.share")
            , col("laureates.motivation"))) # Use dot notion for columns in the STRUCT field
df_flattened.show(5)

+---------+----+-----------------+----+---------+---------+-----+--------------------+
| category|year|overallMotivation|  id|firstname|  surname|share|          motivation|
+---------+----+-----------------+----+---------+---------+-----+--------------------+
|chemistry|2022|             null|1015|  Carolyn| Bertozzi|    3|"for the developm...|
|chemistry|2022|             null|1016|   Morten|   Meldal|    3|"for the developm...|
|chemistry|2022|             null| 743|    Barry|Sharpless|    3|"for the developm...|
|economics|2022|             null|1021|      Ben| Bernanke|    3|"for research on ...|
|economics|2022|             null|1022|  Douglas|  Diamond|    3|"for research on ...|
+---------+----+-----------------+----+---------+---------+-----+--------------------+
only showing top 5 rows



#### 2.4. `get_json_object()` and `json_tuple()` functions

In [18]:
from pyspark.sql.functions import get_json_object

# create a DataFrame with a JSON string column
df3 = spark.createDataFrame([
  (1, '{"name": "Alice", "age": 25}'),
  (2, '{"name": "Bob", "age": 30}')
], ["id", "json_data"])

# extract the "name" field from the JSON string column
name_df = df3.select(get_json_object("json_data", "$.name").alias("name"))
name_df.show()
print("________________________________")
# cast the extracted value to a string
name_str_df = name_df.withColumn("name_str", name_df["name"].cast(StringType()))

name_str_df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
+-----+

________________________________
+-----+--------+
| name|name_str|
+-----+--------+
|Alice|   Alice|
|  Bob|     Bob|
+-----+--------+



### 3. Reading Parquet data

#### 3.1. What is parquet
- Parquet 是一种列式存储格式, 用于存储大规模数据集. Parquet 文件通常以 .parquet 作为文件扩展名, 例如 data.parquet
- Parquet 文件具有高效的存储和查询性能, 并且支持多种数据类型和复杂的数据结构

In [28]:
# Read Parquet file into a DataFrame
parquet_file_location = "../data/dataload/recipes.parquet"
df4 = (spark.read.format("parquet")
      .load(parquet_file_location))
df4.printSchema()

root
 |-- RecipeId: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- AuthorId: integer (nullable = true)
 |-- AuthorName: string (nullable = true)
 |-- CookTime: string (nullable = true)
 |-- PrepTime: string (nullable = true)
 |-- TotalTime: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- RecipeCategory: string (nullable = true)
 |-- Keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- RecipeIngredientQuantities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- RecipeIngredientParts: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- AggregatedRating: double (nullable = true)
 |-- ReviewCount: integer (nullable = true)
 |-- Calories: double (nullable = true)
 |-- FatContent: double (nullable = true)
 |-- SaturatedFatContent: double (nullable = true)
 |-- CholesterolContent: double (nullable = true)
 |-- SodiumContent: double (nullable = true)
 |-- Carbohydr

In [29]:
df4.show(5)

+--------+--------------------+----------+------------+--------+--------+---------+--------------------+--------------+--------------------+--------------------------+---------------------+----------------+-----------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+-----------+--------------------+------+-------------+
|RecipeId|                Name|  AuthorId|  AuthorName|CookTime|PrepTime|TotalTime|         Description|RecipeCategory|            Keywords|RecipeIngredientQuantities|RecipeIngredientParts|AggregatedRating|ReviewCount|Calories|FatContent|SaturatedFatContent|CholesterolContent|SodiumContent|CarbohydrateContent|FiberContent|SugarContent|ProteinContent|RecipeServings|RecipeYield|  RecipeInstructions|Images|DatePublished|
+--------+--------------------+----------+------------+--------+--------+---------+--------------------+--------------+--------------------+----------------

#### 3.2 Partitioned Parquet files

In [30]:
partitioned_parquet_file_location = "../data/dataload/partitioned_recipes"
df_partitioned = (spark.read.format("parquet")
                  .load(partitioned_parquet_file_location))
df_partitioned.show(5)

+--------+--------------------+----------+-------------------+--------+--------+---------+--------------------+--------------+--------------------+--------------------------+---------------------+----------------+-----------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+-----------+--------------------+--------------------+-------------+
|RecipeId|                Name|  AuthorId|         AuthorName|CookTime|PrepTime|TotalTime|         Description|RecipeCategory|            Keywords|RecipeIngredientQuantities|RecipeIngredientParts|AggregatedRating|ReviewCount|Calories|FatContent|SaturatedFatContent|CholesterolContent|SodiumContent|CarbohydrateContent|FiberContent|SugarContent|ProteinContent|RecipeServings|RecipeYield|  RecipeInstructions|              Images|DatePublished|
+--------+--------------------+----------+-------------------+--------+--------+---------+--------------------+---

In [31]:
spark.stop()